# CNV-seq Analysis

This script and helper functions implements an R-only version of
the work flow from https://github.com/hliang/cnv-seq. It uses
GenomicRanges utilities to perform read counts across bins, and the
`cnv` package available at the URL above for additional
analysis. No intermediate files are generates.

This R script has been converted into a jupyter notebook from Bioconductor's RNCV_seq.R code from their copy-number-analysis repository https://github.com/Bioconductor/copy-number-analysis/wiki/CNV-seq.

## <span style="color:red">User Input (MANDATORY)</span>

---
<span style="color:red">**NOTE:**</span>

     Run the cell below once the config-CNVseq.yml file has been edited accordingly.

---

In [ ]:
configPath <- "config/config-CNVseq.yml"

## Installing Required Libraries (run only once)

There are currently two options to install necessary packages: using R or conda. The installation needs to be run the first time the tool is used. Once the installation has run, there is no need for re-installation as the necessary packages will be already installed in the working environment.

### OPTION 1 - Using R 

In [ ]:
# install necessary libraries using R functions
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install(c("GenomicAlignments"))
install.packages("configr")
install.packages("ggplot2")

### OPTION 2 - Using conda

In [ ]:
# install 
system("conda install -y -c conda-forge r-biocmanager -y", intern=TRUE)
system("conda install -y -c bioconda bioconductor-genomicalignments", intern=TRUE)
system("conda install -y -c pcgr r-configr", intern=TRUE)
system("conda install -y -c r r-ggplot2", intern=TRUE)

## Loading Required Libraries

In [ ]:
# load necessary libraries
library(GenomicAlignments)
library(ggplot2)
library(configr)

# specify source R script with helper functions
source("helper-functions/RCNV_seq-helper.R")
source("helper-functions/cnvHLiang.R")

## CNV Calculations and Plotting

In [ ]:
# extract config file fields and values
config <- read.config(configPath)

# store comparisons to be done
comparisons <- config$comparisons
comparisonNames <- names(config$comparisons)

# store chromosomes to subset
chromosomes <- unlist(strsplit(gsub(" ", "", config$chromosomes), ","))

# store provided parameters
parameters <- config$parameters
parameterNames <- names(config$parameters)

# first check if parameter exists. if exist, store value,
# if not exist, use default values
if ('annotate' %in% parameterNames){
    annotate <- config$parameters$annotate
} else {annotate <- TRUE}
if ('bigger' %in% parameterNames){
    bigger <- config$parameters$bigger
} else {bigger <- 1.5}
if ('log2' %in% parameterNames){
    log2 <- config$parameters$log2
} else {log2 <- 0.6}
if ('pvalue' %in% parameterNames){
    pvalue <- config$parameters$pvalue
} else {pvalue <- 0.001}
if ('window_size' %in% parameterNames){
    window_size <- config$parameters$window_size
} else {
    window_size <- windowSize(files, pvalue=pvalue, log2=log2, bigger=bigger)
}

# loop per comparisons 
for (comparison in comparisonNames){
    cat(paste("\nComparison:", comparison))
    cat(paste("\nComparing samples:\n"))
    
    # files to compare
    control <- config$comparisons[[comparison]]$control
    mutant <- config$comparisons[[comparison]]$mutant
    
    cat(paste(control, "\nvs.\n", mutant, "\n"))
    
    # name mutant as "test" and control as "ref"
    files <- file.path(c(mutant, control))
    names(files) <- c("test", "ref")
    
    # calculate overall tiles and hits
    tiles <- tileGenomeOverlap(files, window_size)
    hits <- summarizeOverlaps(tiles, files, binCounter)
    
    # subset only those chromosomes defined in config file
    hitsChrSubset <- subset(hits, seqnames %in% chromosomes)
    
    # create tabulated file with dataframe
    hitsPath <- paste("tab-files/", comparison, "-all-hits.tab", sep="")
    hitsFile <- as.countsfile(hitsChrSubset, hitsPath)

    # calculate Copy Number Variations (CNVs)
    cnv <- cnv.cal(hitsFile, log2=log2, annotate=annotate)
    
    # path to save images in png format
    pngPath <- paste("images/", comparison, "-all-chromosomes.png", sep="")
    png(pngPath)
    
    # plot all chromosomes' CNVs
    plotAll <- plot.cnv.all(cnv, title="Copy Number Variants - All", ylim=c(-5,5), colour=2)
    plotAll <- plotAll + labs(title="CNV - All Chromosomes", 
                              subtitle=comparison, caption=pngPath) +
                         theme(plot.title=element_text(hjust=0.5, size=16),
                               plot.subtitle=element_text(hjust=0.5, size=14), 
                               plot.caption=element_text(hjust=0, face="italic", size=12),
                               axis.text.x=element_text(angle=20, vjust=1, hjust=1))
    print(plotAll)
    
    # save and close pdf device  
    dev.off()
    
    # print figure to stdout
    print(plotAll)
    
    # loop per chromosomes
    for (chrom in chromosomes){
        # subset hits per chromosome defined in config file
        chrSubset <- subset(hits, seqnames %in% chrom)
        chrFilePath <-  paste("tab-files/", comparison, "-", chrom, ".tab", sep="")

        # create tab file per chromosome
        chrSubsetFile <- as.countsfile(chrSubset, chrFilePath)
        
        # calculate CNVs per chromosome
        cnv <- cnv.cal(chrSubsetFile, log2=log2, annotate=annotate)
        
        # path to save images in png format
        pngPath <- paste("images/", comparison, "-chromosome-", chrom, ".png", sep="")
        png(pngPath)
        
        # plot chromosome CNV
        plotChr <- plot.cnv.chr(cnv, chromosome=chrom, title=chrom, ylim=c(-3,3))
        plotChrTitle <- paste("CNV -", chrom)
        plotChr <- plotChr + labs(title=plotChrTitle, 
                                  subtitle=comparison, caption=pngPath) +
                             theme(plot.title=element_text(hjust=0.5, size=14),
                                   plot.subtitle=element_text(hjust=0.5), 
                                   plot.caption=element_text(hjust=0, face="italic"))
        print(plotChr)
        
        # save and close pdf device  
        dev.off()
        
        # print figure to stdout
        print(plotChr)
    }
    
}